In [1]:
import os
import sys

current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, ".."))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
from treemort.utils.config import setup

config_file_path = "../configs/unet_bs8_cs256.txt"
conf = setup(config_file_path)

# Modified Config Variables for Local Execution; comment on HPC
conf.data_folder = "/Users/anisr/Documents/AerialImages"
conf.output_dir = os.path.join("..", conf.output_dir)

print(conf)

Namespace(data_folder='/Users/anisr/Documents/AerialImages', hdf5_file='AerialImageModel_ITD.h5', model='unet', epochs=100, train_batch_size=8, val_batch_size=8, test_batch_size=8, train_crop_size=256, val_crop_size=256, test_crop_size=256, val_size=0.2, test_size=0.1, input_channels=4, output_channels=1, output_dir='../output/unet', model_weights='best', learning_rate=0.0001, threshold=0.5, activation='sigmoid', loss='hybrid', resume=True)


In [4]:
from treemort.main import run

eval_only = True

run(conf, eval_only)

[INFO] Output directory already exists: ../output/unet
[INFO] Using device: cpu
[INFO] Preparing datasets...
[INFO] Datasets prepared: Train(213), Val(61), Test(30)
[INFO] Loading or resuming model...
[INFO] Building model...
[INFO] Configuring model...
[INFO] Unet model configured.
[INFO] Model successfully moved to cpu.
[INFO] Adam optimizer configured with learning rate 0.0001.
[INFO] Model, optimizer, and loss function setup complete.
[INFO] No checkpoint found. Training from scratch.
[INFO] Model, optimizer, criterion, and metrics are set up.
[INFO] Setting up callbacks for 213 batches...
[INFO] Callbacks set up: 4 callbacks configured.
[INFO] Evaluation-only mode started.


Evaluating: 30it [00:20,  1.49it/s, iterations_left=-27]                    

 Mean IOU Pixels: 0.300
 Mean IOU Trees: 0.302
[INFO] Evaluation completed.
